# Module Imports

In [233]:
import pickle
import numpy
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

# Import data

In [768]:
def importData(filename):
    X=[]
    Y=[]
    with open(filename,'r') as data_file:
        counter=0
        for line in data_file:
            if(counter>0):
                counter=counter+1
                line_list=line.split(",")
                if(line_list[0]=='FALSE'):
                    Y.append(line_list[0])
                    X.append([line_list[1],line_list[2],line_list[3].strip("\n")])
                else:
                    Y.append(line_list[0])
                    X.append([line_list[1],line_list[2],line_list[3].strip("\n")])
                    Y.append(line_list[0])
                    X.append([line_list[1],line_list[2],line_list[3].strip("\n")])
                    Y.append(line_list[0])
                    X.append([line_list[1],line_list[2],line_list[3].strip("\n")])
                   
                    
                    
                    
                    
                
            
                
            else:
                counter=counter+1
        
    return X,Y       

In [769]:
X,Y=importData('dataset_2.csv')

In [770]:
X[:10]

[['>10', '<4.4', 'HR|AIRTEL'],
 ['>5', '<5', 'UP|JIO'],
 ['>10', '<5', 'AP|AIRTEL'],
 ['>5', '<5', 'RJ|JIO'],
 ['>10', '<5.6', 'WB|VODAFONE'],
 ['>10', '<5.6', 'HR|JIO'],
 ['>10', '<5.6', 'HR|JIO'],
 ['>10', '<5.6', 'HR|JIO'],
 ['>5', '<5', 'TN|JIO'],
 ['>20', '<6', 'DL|AIRTEL']]

# Inspect X1 and X2 Features

In [771]:
x1=[]
x2=[]
x1=set(x1)
x2=set(x2)
for i in range(len(X)):
    x1.add(float(X[i][0][1:]))
    x2.add(float(X[i][1][1:]))

In [772]:
x1

{5.0, 10.0, 20.0, 30.0, 40.0}

In [773]:
x2

{4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0}

# One Hot Encoding

In [774]:
print('Creating Vectorizer for whole data set')
vectorizer = DictVectorizer(sparse=False)
vectorizer_input=[]
for eachRow in X:
    temp_dic={}
    temp_count=1
    for data in eachRow:
        temp_dic['X'+str(temp_count)] = data
        temp_count=temp_count+1
    vectorizer_input.append(temp_dic)
print("Dictonary complete")
output_all = vectorizer.fit(vectorizer_input)


Creating Vectorizer for whole data set
Dictonary complete


In [775]:
vectorizer_input[3000]

{'X1': '>10', 'X2': '<5', 'X3': 'KL|IDEA'}

In [776]:
len(vectorizer_input)

4323

In [777]:
from random import shuffle
shuffle(vectorizer_input)

# Train test split

In [778]:
train_X=vectorizer_input[:3500]
train_Y=Y[:3500]
test_X=vectorizer_input[3500:]
test_Y=Y[3500:]

# Serialize the one hot encoder

In [779]:
with open('dictvectorizer.pkl', 'wb') as fid:
    pickle.dump(vectorizer, fid)

In [780]:
vectorizer.transform(vectorizer_input[0]).shape

(1, 177)

# Train a logistic regression model

In [781]:
from sklearn import linear_model

In [782]:
clf = linear_model.LogisticRegression(C=1e5)
clf.fit(vectorizer.transform(train_X),train_Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [783]:
predictions=clf.predict(vectorizer.transform(test_X)).tolist()

In [784]:
len(predictions)

823

In [785]:
from sklearn.metrics import accuracy_score
accuracy_score(test_Y,predictions)

0.70109356014580804

In [756]:
import pandas
from pandas_ml import ConfusionMatrix

import matplotlib.pyplot as plt

cm = ConfusionMatrix(y_true=test_Y, y_pred=predictions)
cm
cm.print_stats()

population: 823
P: 219
N: 604
PositiveTest: 108
NegativeTest: 715
TP: 24
TN: 520
FP: 84
FN: 195
TPR: 0.109589041096
TNR: 0.860927152318
PPV: 0.222222222222
NPV: 0.727272727273
FPR: 0.139072847682
FDR: 0.777777777778
FNR: 0.890410958904
ACC: 0.6609963548
F1_score: 0.146788990826
MCC: -0.0385886141332
informedness: -0.0294838065862
markedness: -0.0505050505051
prevalence: 0.26609963548
LRP: 0.787997390737
LRN: 1.03424657534
DOR: 0.761904761905
FOR: 0.272727272727


In [787]:
cm

Predicted  FALSE  TRUE  __all__
Actual                         
FALSE        520    84      604
TRUE         195    24      219
__all__      715   108      823